# Scraping the Dark Web
The ransomware websites that we have chosen to scrape are the following:
1) REvil
2) Conti
3) Pysa
4) XINGTeam
5) BlackMatter
6) RansomExx

Using BeautifulSoup and Selenium, names of victim companies have been scraped from the dark web, and their respective industries and headquarter locations are scraped from the clear web using Selenium.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.request import urlopen
from geopy.geocoders import Nominatim 

import socks
import socket
import requests
import pandas as pd
import numpy as np
import time

In [2]:
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
old_socket = socket.socket # saving old socket info 
socket.socket = socks.socksocket
def getaddrinfo(*args):
    return [(socket.AF_INET, socket.SOCK_STREAM, 6, '', (args[0], args[1]))]
socket.getaddrinfo = getaddrinfo

## 1) Scraping REvil

In [3]:
REvil_link = "http://dnpscnbaix6nkwvystl3yxglz7nteicqrou3t75tpcc5532cztc46qyd.onion"
res = requests.get(REvil_link)
soup = BeautifulSoup(res.content, 'html.parser')

In [7]:
# Putting all weblinks into a list 
links = [link.get('href') for link in soup.find_all('a', href=True)]
links = list(filter(None, links))

webpage_links = [] 
for l in links:
    if "page" in l:
        l = REvil_link + l
        webpage_links.append(l)
        
webpage_links = webpage_links[1:-1] # remvoving first and last element from list

In [8]:
# Getting list of victims from REvil
REvilHeader_list = []
for link in webpage_links:
    res = requests.get(link)
    soup = BeautifulSoup(res.content, 'html.parser')
    posts = soup.find_all(class_ = "blog-post-title")
    
    for post in posts:
        title = post.find('a')
        REvilHeader_list.append(title.text)

In [10]:
# Placing victims into a DataFrame 
df = pd.DataFrame(columns=['Company Name', 'Industry', 'Headquarters'])

In [ ]:
# Using Selenium to automate finding industries
socket.socket = old_socket # reuse the original socket for selenium
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()

In [ ]:
# Glassdoor search
company_list = []
industry_list = []
location_list = []
number = 0

for header in REvilHeader_list: 
    try:
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(header + ' glassdoor overview')
        search.send_keys(Keys.RETURN)
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:
            company = driver.find_element_by_tag_name('h1')
            company_list.append(company.text)
            
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    industry_list.append(industry)
                    break
            else:
                industry_list.append(np.nan)
                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    location_list.append(location)
                    break
            else:
                location_list.append(np.nan)
        else:
            company_list.append(np.nan)
            industry_list.append(np.nan)
            location_list.append(np.nan)
    except Exception as e:
        company_list.append(np.nan)
        industry_list.append(np.nan)
        location_list.append(np.nan)
        print(e)

    print(str(number) + ") " + header)
    number = number + 1
    
driver.quit()

In [ ]:
# Placing all scraped data into a DataFrame
df['Company Name'] = company_list
df['Industry'] = industry_list
df['Headquarters'] = location_list

# Cleaning data
df = df.dropna()
df = df.reset_index(drop=True) # reset index

df.to_csv('REvil.csv')

In [ ]:
# Finding countries of headquarters
data = pd.read_csv('Revil.csv')
geolocator = Nominatim(user_agent = "geoapiExercises")
country_list = []

for city in data['Headquarters']:
    place, (lat, lng) = geolocator.geocode(city)
    country = place.split()[-1]
    country_list.append(country)
    
data['Country'] = country_list
df.to_csv('REvil.csv')

## 2) Scraping Conti

In [11]:
conti_link = "http://continewsnv5otx5kaoje7krkto2qbu3gtqef22mnr7eaxw3y6ncz3ad.onion/"
res = requests.get(conti_link)
soup = BeautifulSoup(res.content, 'html.parser')

In [12]:
# Manually creating the links as the scraping of links have some issue
conti_link_manual = "http://continewsnv5otx5kaoje7krkto2qbu3gtqef22mnr7eaxw3y6ncz3ad.onion/page/"
webpage_links = [] 
for x in range (1,54):
    str_convert = str(x)
    final = conti_link_manual + str_convert
    webpage_links.append(final)

In [ ]:
# Find all company that are affected with conti ransomware
company_list = []
for link in webpage_links:
    res = requests.get(link)
    soup = BeautifulSoup(res.content, 'html.parser')
    posts = soup.findAll("div", {"class": "title"})
    for post in posts:
        l = post.text
        cleaned = l.replace('“', '').replace('”','')
        company_list.append(cleaned)

In [ ]:
# Find all dates that company was affected with ransomware 
date_list = []
for link in webpage_links:
    res = requests.get(link)
    soup = BeautifulSoup(res.content, 'html.parser')
    posts = soup.findAll("div", {"class": "footer"})
    for post in posts:
        l = post.text
        split_string = l.split(",", 1)
        substring = split_string[0]
        cleaned = substring.replace('\n','')
        date_list.append(cleaned)

In [ ]:
# Placing victims into a DataFrame
df = pd.DataFrame(columns=['Company Name', 'Industry', 'Affected_Date'])
df['Company Name'] = company_list
df['Affected_Date'] = date_list

In [ ]:
# Using Selenium to automate finding industries
socket.socket = old_socket # reuse the original socket for selenium
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()

In [ ]:
# glassdoor search
industry_list = []
location_list = []
number = 0

for header in df['Company name']: 
    try:
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(header + ' glassdoor overview')
        search.send_keys(Keys.RETURN)
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:     
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    industry_list.append(industry)
                    break
            else:
                industry_list.append(np.nan)
                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    location_list.append(location)
                    break
            else:
                location_list.append(np.nan)
        else:
            industry_list.append(np.nan)
            location_list.append(np.nan)
    except Exception as e:
        industry_list.append(np.nan)
        location_list.append(np.nan)
        print(e)

    print(str(number) + ") " + header)
    number = number + 1
    
driver.quit()

In [ ]:
# Placing all scraped data into a DataFrame
df['Industry'] = industry_list
df['Headquarters'] = location_list

# Cleaning data
df = df.dropna()
df = df.reset_index(drop=True) # reset index

df.to_csv('Conti.csv')

In [ ]:
# Finding countires of headquarters
data = pd.read_csv('Conti.csv')
geolocator = Nominatim(user_agent = "geoapiExercises")
country_list = []

for city in data['Headquarters']:
    place, (lat, lng) = geolocator.geocode(city)
    country = place.split()[-1]
    country_list.append(country)
    
data['Country'] = country_list
df.to_csv('Conti.csv')

## 3) Scraping Pysa

In [ ]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import os

In [ ]:
# Navigating to Pysa
os.popen(r'/home/kali/.local/share/torbrowser/tbb/x86_64/tor-browser_en-US/Browser/TorBrowser/Tor/tor')

binary=FirefoxBinary(r'/home/kali/.local/share/torbrowser/tbb/x86_64/tor-browser_en-US/Browser/firefox')
fp=FirefoxProfile(r'/home/kali/.local/share/torbrowser/tbb/x86_64/tor-browser_en-US/Browser/TorBrowser/Data/Browser/profile.default/')

fp.set_preference('extensions.torlauncher.start_tor',False)#note this
fp.set_preference('network.proxy.type',1)
fp.set_preference('network.proxy.socks', '127.0.0.1')
fp.set_preference('network.proxy.socks_port', 9150)
fp.set_preference("network.proxy.socks_remote_dns", True)
fp.update_preferences()

driver = webdriver.Firefox(firefox_profile=fp,firefox_binary=binary)
driver.get('http://pysa2bitc5ldeyfak4seeruqymqs4sj5wt5qkcq7aoyg4h2acqieywad.onion/partners.html')

In [ ]:
# Obtaining vitcims from Pysa
elements = driver.find_elements_by_class_name('page-header')
companyList = []
webList = []
for i in elements:
    element = i.find_elements_by_xpath('.//span')
    companyList.append(element[0].text)
    webList.append(element[2].text)

In [ ]:
# Placing Pysa victims into a DataFrame
df = pd.DataFrame({"Company Name":companyList, "Website": webList})
df['Industry'] = np.nan
df['Location'] = np.nan
df.to_csv("Pysa.csv")

In [ ]:
# Using Selenium to automate finding of industries and headquarters
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
old_socket = socket.socket # saving old socket info 
socket.socket = socks.socksocket
socket.socket = old_socket # reuse the original socket for selenium
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()

In [ ]:
# Glassdoor search
industry_list = []
location_list = []
number = 0

for i in df["Company Name"]: 
    try:
        driver.get("https://www.google.com/search?q=" + i + " glassdoor overview")
        
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    industry_list.append(industry)
                    break
            else:
                industry_list.append(np.nan)
                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    location_list.append(location)
                    break
            else:
                location_list.append(np.nan)
        else:
            company_list.append(np.nan)
            industry_list.append(np.nan)
            location_list.append(np.nan)
    except Exception as e:
        company_list.append(np.nan)
        industry_list.append(np.nan)
        location_list.append(np.nan)
        print(e)

    print(str(number) + ") " + i + " | " + industry + " | " + location)
    number = number + 1
    time.sleep(2)

In [ ]:
# Placing scraped data into DataFrame
df['Industry'] = industry_list
df['Headquarters'] = location_list

df.to_csv('PysaWithNA.csv')

In [ ]:
# Using selenium to automate scraping of industries and headquarters using LinkedIn
socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)
socket.socket = socket.socket
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)

In [ ]:
# Login to LinkedIn with throwaway account
email = "lando.ozan@zooants.com"
password = "password123"

driver.get('https://www.linkedin.com/login')
time.sleep(3)

driver.find_element_by_id('username').send_keys(email)
driver.find_element_by_id('password').send_keys(password)
driver.find_element_by_id('password').send_keys(Keys.RETURN)

In [ ]:
# LinkedIn search
index = 0
x = np.nan

for company in df["Company Name"]: 
    if ( pd.isnull(df.loc[index, "Industry"]) or pd.isnull(df.loc[index, "Headquarters"])):
        print(company)
        try:
            driver.get("https://www.google.com/search?q=" + company + " linkedin")
            results = driver.find_elements_by_css_selector('div.g')
            link = results[0].find_element_by_tag_name("a")
            href = link.get_attribute("href")
            driver.get(href + "/about") 
        except:
            print("Access Error")
            continue
        
        if (pd.isnull(df.loc[index, "Industry"])):
            try:
                industry = driver.find_element_by_xpath('//dt[text()="Industry"]/following-sibling::dd')
                df.loc[index, "Industry"] = industry.text
            except:
                df.loc[index, "Industry"] = "NA"
        if (pd.isnull(df.loc[index, "Headquarters"])):
            try:
                hq = driver.find_element_by_xpath('//dt[text()="Headquarters"]/following-sibling::dd')
                df.loc[index, "Headquarters"] = hq.text
            except:
                df.loc[index, "Headquarters"] = "NA"
    

    index = index + 1    

In [ ]:
# Saving DataFrame to CSV file
df.to_csv('Pysa.csv')

# Finding countires of headquarters
data = pd.read_csv('Pysa.csv')
geolocator = Nominatim(user_agent = "geoapiExercises")
country_list = []

for city in data['Headquarters']:
    place, (lat, lng) = geolocator.geocode(city)
    country = place.split()[-1]
    country_list.append(country)
    
data['Country'] = country_list
df.to_csv('Pysa.csv')

## 4) Scraping XINGTeam

In [ ]:
# Accessing XINGTeam link
XING_link = "http://xingnewj6m4qytljhfwemngm7r7rogrindbq7wrfeepejgxc3bwci7qd.onion"
res = requests.get(XING_link)
soup = BeautifulSoup(res.content, 'html.parser')

In [ ]:
# Getting all links out of the soup and deleting None's

links = [link.get('href') for link in soup.find_all('a', href=True)]
links = list(filter(None, links))

webpage_links = [] 
for l in links:
    l = XING_link + l
    webpage_links.append(l)
        
webpage_links = webpage_links[1:-1] # remvoving first and last element from list

In [ ]:
# Getting all victim companies
info_list = []

for link in webpage_links:
    row_info = []
    res = requests.get(link)
    soup = BeautifulSoup(res.content, 'html.parser')
    dt_tags = soup.find_all('dt')
    dd_tags = soup.find_all('dd')

    for tag in dd_tags:
        row_info.append(tag.text)
    info_list.append(row_info)

info_list

In [ ]:
# Placing scraped data into a DataFrame
df = pd.DataFrame(info_list)
df.columns = ['Company Name', 'Total Revenue', 'Volume of Stolen Data', 'Published', 'Last Updated', 'Info']

In [ ]:
# Automating finding of industries and headquarters using Selenium
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()

In [ ]:
# Glassdoor search
company_list = []
industry_list = []
location_list = []
number = 0

for company in df['Company Name']: 
    try:
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(company + ' glassdoor "overview"')
        search.send_keys(Keys.RETURN)
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    industry_list.append(industry)
                    break
            else:
                industry_list.append(np.nan)
                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    location_list.append(location)
                    break
            else:
                location_list.append(np.nan)
        else:
            industry_list.append(np.nan)
            location_list.append(np.nan)
    except Exception as e:
        industry_list.append(np.nan)
        location_list.append(np.nan)
        print(e)

    print(str(number) + ") " + company)
    number = number + 1
    
driver.quit()

In [ ]:
# Storing data in DataFrame + Cleaning data
new_df = pd.DataFrame(columns=['Company Name', 'Industry', 'Headquarters', 'Date of attack'])

new_df['Company Name'] = df['Company Name']
new_df['Industry'] = industry_list
new_df['Headquarters'] = location_list
new_df['Date of attack'] = df['Published']

new_df = new_df.dropna()
new_df = new_df.reset_index(drop=True) # reset index

new_df.to_csv('XINGTeam.csv')

In [ ]:
# Finding countries of victim companies
data = pd.read_csv('XINGTeam.csv')

geolocator = Nominatim(user_agent = "geoapiExercises")
num = 0
country_list = []

for city in data['Headquarters']:
    place, (lat, lng) = geolocator.geocode(city)
    country = place.split()[-1]
    country_list.append(country)
    
data['Country'] = country_list

data.to_csv("XINGTeam_Final.csv")

## 5) Scraping BlackMatter

In [ ]:
# Accessing BlackMatter
blackmatter_link = "http://blackmax7su6mbwtcyo3xwtpfxpm356jjqrs34y4crcytpw7mifuedyd.onion/"
res = requests.get(blackmatter_link)
soup = BeautifulSoup(res.content, 'html.parser')

In [ ]:
# Getting all victim companies
blackmatter_links = ["http://blackmax7su6mbwtcyo3xwtpfxpm356jjqrs34y4crcytpw7mifuedyd.onion/",
                    "http://blackmax7su6mbwtcyo3xwtpfxpm356jjqrs34y4crcytpw7mifuedyd.onion/?page=2&per-page=18"]

header_list = []

for link in blackmatter_links:
    res = requests.get(link)
    soup = BeautifulSoup(res.content, 'html.parser')
    h4_tags = soup.find_all('h4')

    for tag in h4_tags:
        header_list.append(tag.text)

In [ ]:
# Saving data into DataFrame
df = pd.DataFrame(columns=['BlackMatter Headers', 'Company Name', 'Industry', 'Headquarters'])
df['BlackMatter Headers'] = header_list

df.to_csv('BlackMatter.csv')

In [ ]:
# Using Selenium to automate fidning of industries + headquarters
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.maximize_window()

In [ ]:
# Glassdoor search
company_list = []
industry_list = []
location_list = []
number = 0

for header in header_list: 
    try:
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(header + ' glassdoor "overview"')
        search.send_keys(Keys.RETURN)
        glassdoor_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        glassdoor_page.click()
        
        if "glassdoor" in driver.current_url:
            company = driver.find_element_by_tag_name('h1')
            company_list.append(company.text)
            
            pagesource = driver.page_source
            soup = BeautifulSoup(pagesource, 'html.parser')
            labels = soup.find_all('label')
            for label in labels:
                if (label.text == "Industry:"):
                    industry = label.next_sibling.text
                    industry_list.append(industry)
                    break
            else:
                industry_list.append(np.nan)
                
            for label in labels:
                if (label.text == "Headquarters:"):
                    location = label.next_sibling.text
                    location_list.append(location)
                    break
            else:
                location_list.append(np.nan)
        else:
            company_list.append(np.nan)
            industry_list.append(np.nan)
            location_list.append(np.nan)
    except Exception as e:
        company_list.append(np.nan)
        industry_list.append(np.nan)
        location_list.append(np.nan)
        print(e)

    print(str(number) + ") " + header)
    number = number + 1
    
driver.quit()

In [ ]:
# Storing scraped data into DataFrame + Cleaning data
df['BlackMatter Headers'] = header_list
df['Company Name'] = company_list
df['Industry'] = industry_list
df['Headquarters'] = location_list
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True) # reset index
df

df.to_csv('BlackMatter.csv')

In [ ]:
# Finding countries of victim companies
data = pd.read_csv('BlackMatter.csv')

geolocator = Nominatim(user_agent = "geoapiExercises")
num = 0
country_list = []

for city in data['Headquarters']:
    place, (lat, lng) = geolocator.geocode(city)
    country = place.split()[-1]
    country_list.append(country)
    
data['Country']= country_list
data.to_csv("BlackMatter.csv")